In [24]:
import torch
from pydantic import (
    BaseModel,
    Field,
    annotated_types,
    parse_obj_as,
    validator,
    root_validator,
)
from typing import Annotated
import torchvision
import json
import torch.nn as nn
from torchvision.models.detection.faster_rcnn import (
    FastRCNNPredictor,
    fasterrcnn_resnet50_fpn,
)


# dinov2_vits14 = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14")

In [25]:
from courtvision.data import BallDataSet

with open(
    "/Users/benjamindecharmoy/projects/courtvision/datasets/annotations/project-1-at-2023-04-22-11-17-eb4e8e32.json",
    "r",
) as fp:
    dataset = BallDataSet(samples=json.load(fp))

In [18]:
dataset.samples[0].data

AnnotationDataPath(image=PosixPath('/data/upload/1/83dcc83f-curated_001_frame_0001.png'))

In [20]:
from typing import Callable, Optional
from torchvision.datasets import VisionDataset
import matplotlib.pyplot as plt
from courtvision.vis import load_image


class BallDataset(VisionDataset):
    def __init__(
        self,
        dataset: BallDataSet,
        root: str,
        transforms: Callable | None = None,
        transform: Callable | None = None,
        target_transform: Callable | None = None,
    ):

        self.dataset = dataset

    def __len__(self):
        return len(self.dataset.annotations)

    def __getitem__(self, idx):
        annotation = self.dataset.annotations[idx]

    @staticmethod
    def show_sample(sample: CourtAnnotatedSample):
        """Plots an image and its annotation"""
        image = load_image(sample.data.image)
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        plt.scatter(
            sample["annotation"][:, 0],
            sample["annotation"][:, 1],
            marker="x",
            s=100,
            c="r",
        )
        plt.axis("off")
        plt.show()

In [ ]:
head = FastRCNNPredictor(384, 2)

In [ ]:
patch_embed, mod_list = list(dinov2_vits14.children())[:-2]
patch_embed, mod_list

list(mod_list.children())[1:2]

[NestedTensorBlock(
   (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
   (attn): MemEffAttention(
     (qkv): Linear(in_features=384, out_features=1152, bias=True)
     (attn_drop): Dropout(p=0.0, inplace=False)
     (proj): Linear(in_features=384, out_features=384, bias=True)
     (proj_drop): Dropout(p=0.0, inplace=False)
   )
   (ls1): LayerScale()
   (drop_path1): Identity()
   (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
   (mlp): Mlp(
     (fc1): Linear(in_features=384, out_features=1536, bias=True)
     (act): GELU(approximate='none')
     (fc2): Linear(in_features=1536, out_features=384, bias=True)
     (drop): Dropout(p=0.0, inplace=False)
   )
   (ls2): LayerScale()
   (drop_path2): Identity()
 )]

In [ ]:
head(nn.Sequential(*[patch_embed, *mod_list[:]])(torch.rand(1, 3, 224, 224))).shape

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x98304 and 384x2)

In [ ]:
fast_model = fasterrcnn_resnet50_fpn(
    pretrained=True,
    num_classes=91,
)
fast_model

/Users/benjamindecharmoy/projects/courtvision/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/benjamindecharmoy/projects/courtvision/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
from courtvision.vis import load_timg

image = load_timg(
    "/Users/benjamindecharmoy/projects/courtvision/data/frames/curated_001/frame_0001.png"
)
with torch.no_grad():
    # fast_model.backbone.body = nn.Sequential(*[patch_embed, *mod_list[:]])
    # fast_model.roi_heads.box_predictor = head
    fast_model.eval()
    out = fast_model(image)

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
with torch.no_grad():
    # fast_model.backbone.body = nn.Sequential(*[patch_embed, *mod_list[:]])
    # fast_model.roi_heads.box_predictor = head
    model.eval()
    out = model(image)

In [ ]:
import rerun as rr
from rerun.components.rect2d import RectFormat

rr.init("faster", spawn=True)

In [ ]:
rr.log_rigid3(entity_path="frame/test", parent_from_child=([0, 10, 0], [0, 111, 0, 10]))

In [ ]:
rr.log_image("frame/test/img", image.squeeze(0).permute(1, 2, 0))
# out[0]["boxes"]

In [ ]:
dd = [box.tolist() for box in out[0]["boxes"]]
rr.log_rects("frame/boxes", dd, rect_format=RectFormat.XYXY)

In [ ]:
from rerun.components.rect2d import RectFormat

rr.log_rect("frame/box", out[0]["boxes"][2].tolist(), rect_format=RectFormat.XYXY)

In [ ]:
out[0]["boxes"][0].tolist()

[586.6439819335938, 360.8269348144531, 1062.5076904296875, 670.1864624023438]

In [ ]:
image.shape

torch.Size([1, 3, 720, 1280])

In [ ]:
out[0]

{'boxes': tensor([[ 586.6440,  360.8269, 1062.5077,  670.1865],
         [ 597.6040,  191.3133,  617.9384,  211.2779],
         [ 297.4283,  581.7656, 1115.0757,  708.7494],
         [ 389.4426,  128.8055,  970.5952,  490.9042],
         [ 434.5761,  284.2084,  946.3016,  621.3353],
         [ 761.4454,  195.3744,  802.9850,  293.6284],
         [ 428.8613,  451.5882,  473.8964,  573.3542],
         [ 821.0404,  284.2837, 1154.2217,  720.0000],
         [ 898.5524,  426.9127,  946.2028,  537.3666],
         [ 416.9436,  498.2014, 1103.0070,  664.2693],
         [ 878.7841,  415.2511,  968.8537,  574.6574],
         [ 757.9148,   17.3750,  977.1315,  369.8183],
         [ 763.1102,  232.0803,  817.0454,  287.9791],
         [ 573.9623,  174.1141,  595.5928,  222.5417],
         [ 588.5248,  189.3533,  609.8690,  209.1296],
         [ 406.5399,   64.9292,  815.7542,  349.9398],
         [ 471.6840,   19.7566,  897.4198,  720.0000],
         [1229.1489,  222.3493, 1280.0000,  271.0620],
 

In [ ]:
import label_studio

ModuleNotFoundError: No module named 'label_studio'